In [1]:
import pandas as pd
from sklearn import preprocessing

df1=pd.read_csv("../data/raw/Students_Result_Data.csv")

# Loop to check the unique values in original dataframe
for i in df1.columns:
    print(df1[i].value_counts())
    print()


School of Engineering             27900
School of Humanities & Soc Sci     4589
School of Natural Sciences         4352
School of Mgnt & Entrepre          1454
Name: School, dtype: int64

Bachelor of Technology            27900
Bachelor of Science (Research)     6358
Bachelor of Arts (Research)        2583
Bachelor of Management Studies     1454
Name: Program, dtype: int64

Mechanical Engineering                       7229
Computer Science and Engineering             7099
Electronics and Communication Engineering    6627
Electrical and Electronics Engineering       3971
Economics                                    2006
Civil Engineering                            1814
English                                      1544
Physics                                      1532
Bachelor of Management Studies               1454
Biotechnology                                1309
Mathematics                                  1122
Chemical Engineering                          984
Sociology              

In [2]:
df1.head()

,School,Program,Major,Semester,Course,Course Name,Credits,Grade
0,School of Engineering,Bachelor of Technology,Electrical Engineering,Monsoon 2012,CCC303,"Literature, Identity and Theatre 2",1.5,B
1,School of Engineering,Bachelor of Technology,Electrical Engineering,Monsoon 2012,CHY101,Applied Chemistry,5.0,C-
2,School of Engineering,Bachelor of Technology,Electrical Engineering,Monsoon 2012,CSD101,Problem Solving through Programming,4.0,D
3,School of Engineering,Bachelor of Technology,Electrical Engineering,Monsoon 2012,CCC302,"Literature, Identity and Theatre 1",1.5,B-
4,School of Engineering,Bachelor of Technology,Electrical Engineering,Monsoon 2012,PHY101,Introduction to Physics I,4.0,F


In [3]:
# Data preprocessing


#there are two departments: So changing electrical engineering to Electrical and Electronics Engineering only
df1['Major'].replace({'Electrical Engineering':'Electrical and Electronics Engineering'},inplace=True)

#drop summer semester
for i,row in df1.iterrows():
    if row['Semester']=='Summer':
        df1.drop(i,inplace=True)


# drop column program and course name
df=df1.drop(columns=['Program'])
df=df.drop(columns=['Course Name'])


# break semester and year in two columns(but keep the original column as well)
# rename semester column
df.rename(columns={'Semester':'Sem_year'}, inplace=True)
idx=list(df.columns).index('Sem_year')+1
sem_col=[]
year_col=[]
sem_col=[i.split(' ',1)[0] for i in df['Sem_year']]
year_col=[i.split(' ',1)[1] for i in df['Sem_year']]
df.insert(loc=idx,column='Semester',value=sem_col)
df.insert(loc=idx+1,column='Year',value=year_col)



#making a batch column
idx=list(df.columns).index('Year')+1
batches=[]
b=df.loc[0][4]
for i in range(0,len(df["Year"])):
    batches.append(b)
    if(i+1<=len(df["Year"])-1):
        if(df.loc[i][4]>df.loc[i+1][4]):
            b=df.loc[i+1][4]
df.insert(loc=idx,column='Batch',value=batches)


In [4]:
df.head()

,School,Major,Sem_year,Semester,Year,Batch,Course,Credits,Grade
0,School of Engineering,Electrical and Electronics Engineering,Monsoon 2012,Monsoon,2012,2012,CCC303,1.5,B
1,School of Engineering,Electrical and Electronics Engineering,Monsoon 2012,Monsoon,2012,2012,CHY101,5.0,C-
2,School of Engineering,Electrical and Electronics Engineering,Monsoon 2012,Monsoon,2012,2012,CSD101,4.0,D
3,School of Engineering,Electrical and Electronics Engineering,Monsoon 2012,Monsoon,2012,2012,CCC302,1.5,B-
4,School of Engineering,Electrical and Electronics Engineering,Monsoon 2012,Monsoon,2012,2012,PHY101,4.0,F


In [5]:
# Proprocessing Continued...

#Map Major names to the major code. Thus, d is a dictionary that can be used later on.
df2 = df1[["Course","Course Name"]]
d = dict()      #d is the dictionary that contains maps the course code against the course name
for i,row in df2.iterrows():
    d[row[0]] = row[1]



#Making a dictionary, mapping Major course names to their codes
majorNames = ['Mechanical Engineering', 'Computer Science and Engineering',
       'Electronics and Communication Engineering',
       'Electrical and Electronics Engineering', 'Economics',
       'Civil Engineering', 'English', 'Physics',
       'Bachelor of Management Studies', 'Biotechnology', 'Mathematics',
       'Chemical Engineering', 'Sociology', 'Chemistry', 'History']
majorCodes = ["MED","CSD", "ECE","EED","ECO","CED","ENG","PHY","BMS","BIO","MAT","CHD","SOC","CHY","HIS"]
majorDict = dict()
for i in range(len(majorNames)):
    majorDict[majorNames[i]] = majorCodes[i]
print(majorDict)

{'Mechanical Engineering': 'MED', 'Computer Science and Engineering': 'CSD', 'Electronics and Communication Engineering': 'ECE', 'Electrical and Electronics Engineering': 'EED', 'Economics': 'ECO', 'Civil Engineering': 'CED', 'English': 'ENG', 'Physics': 'PHY', 'Bachelor of Management Studies': 'BMS', 'Biotechnology': 'BIO', 'Mathematics': 'MAT', 'Chemical Engineering': 'CHD', 'Sociology': 'SOC', 'Chemistry': 'CHY', 'History': 'HIS'}


In [6]:
# Proprocessing Continued...

#Removing unwanted Rows 

#replacing major column values with major codes using the dictionary made above
for k,v in majorDict.items():
    df['Major'].replace({k:v},inplace=True)
    
#dropping all the major courses for each student(since we are working with UWEs and CCCs recommendations only thus reducing size)
for i,row in df.iterrows():     
    if row['Course'].startswith(row['Major']):
        df.drop(i,inplace=True)
        
#for both EEE and ECE department, major courses start with EED only.
for i,row in df.iterrows():     
    if row['Major'].startswith('ECE') and row['Course'].startswith('EED'):
        df.drop(i,inplace=True)
        
#dropping old ccc whihc are no longer offered 
for i,row in df.iterrows():     
    if row['Course']=='CCC090':
        df.drop(i,inplace=True)
        
#dropping grades that are not applicable now and were used 6 years ago      
for i,row in df.iterrows():     
    if row['Grade']=='S' or row['Grade']=='U'or row['Grade']=='Z':
        df.drop(i,inplace=True)
        
#dropping grades that take attendance in consideration     
df['Grade'].replace({'A-*':'A-','B-*':'B-','D*':'D','F*':'F','B*':'B','C*':'C','C-*':'C-'},inplace=True)
#print(df['Grade'].value_counts().keys())
df.dropna(how='any', inplace=True) 


#remove the compulsory courses whose names don't match the major names
# Removing compulsory courses as they dont add value to recommendation as they are compulsory for
# whatever Major subject a student is enrolled into
remove_compul_courses={
#SOE
'MED':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','CED102','MAT102','MAT210','MAT203','MAT205','PHY101','CSD101','EED101'],
'CSD':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','MAT201','MAT203','PHY101','PHY102','EED201','EED101','EED206'],
'ECE':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','CSD201','MAT201','MAT203','MAT205','PHY101','PHY102','CSD101'],
'EED':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','CSD201','MAT201','MAT203','MAT205','PHY101','PHY102','CSD101'],
'CED':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','PHY101','PHY102','MAT205','CSD101','MAT201','MAT203','EED101'],
'CHD':['CHY101','CED101','CED102','MAT101','BIO113','MED201','MED202','MAT102','MAT202','MAT201','PHY101','BIO113','CSD101','MAT203','MAT205','PHY102','MED201','MED101'],

#snss
'MAT':['CSD101'],
'BIO':['MAT020','CHY111','MAT084','CHY112','PHY108','PHY255'],
'CHY':['PHY105','PHY103','MAT101'],
'PHY':['MAT101','CHY111','BIO113','MAT102'],

#shss
'ENG':['ECO108','SOC102'],
'ECO':['ENG104','MAT101','MAT284','SOC102'],
'HIS':['ENG104','SOC102','ECO108'],
'SOC':['ENG104','SOC102','ECO108'],
'BMS':['CSD101']
}

#remove_compul_courses=[[MED],[CSD],[ECE],[EED],[CED],[CHD],[MAT],[BIO],[CHY],[PHY],[ENG],[ECO],[HIS],[SOC],[BMS]]
for i,row in df.iterrows(): 
    for k,v in remove_compul_courses.items():
        if row['Major']==k:
            for value in v:
                if row['Course']==value:
                    df.drop(i,inplace=True)
                    break
            break
            
print(df.head(0))
#on running the loop below, we see overall entries in each column have decreased.
for i in df.columns:
    print(df[i].value_counts())
df = df.reset_index(drop = True)

df.to_csv('../data/preprocessed/Final_data.csv',sep=',',encoding='utf-8',header=True)

Empty DataFrame
Columns: [School, Major, Sem_year, Semester, Year, Batch, Course, Credits, Grade]
Index: []
School of Engineering             11159
School of Humanities & Soc Sci     2223
School of Natural Sciences         1733
School of Mgnt & Entrepre          1447
Name: School, dtype: int64
MED    3060
CSD    2871
ECE    2587
EED    1563
BMS    1447
ECO     986
ENG     745
CED     667
PHY     637
BIO     449
MAT     429
SOC     413
CHD     411
CHY     218
HIS      79
Name: Major, dtype: int64
Spring 2018     4492
Monsoon 2017    4239
Spring 2017     3048
Monsoon 2016    2587
Spring 2016     1107
Monsoon 2015     862
Monsoon 2014      72
Spring 2015       68
Monsoon 2013      20
Summer 2016       18
Spring 2014       14
Monsoon 2012       8
Spring 2013        8
Summer 2017        8
Summer 2015        7
Summer 2018        3
Summer 2014        1
Name: Sem_year, dtype: int64
Spring     8737
Monsoon    7788
Summer       37
Name: Semester, dtype: int64
2017    7295
2018    4495
2016    37

In [7]:
# Now arranging the data in a format where each student has a unique entry in the data frame.

# function which makes a finalList for a student containing values: sem1 courses list, sem1 grades list, sem2 courses list, sem2 grades list,..., sem8 grades list.
# courses are all courses taken by student till date.
# grades are all grades obtained by student till date.
def parseIntoSemesters(courses,grades):
    
    MasterListCourses = []
    SemesterWiseCourses = []
    for i in courses:
        if i=="0":
            MasterListCourses.append(SemesterWiseCourses)
            SemesterWiseCourses = []
        else:
            SemesterWiseCourses.append(i)
    
    MasterListGrades = []
    SemesterWiseGrades = []
    for i in grades:
        if i=="0":
            MasterListGrades.append(SemesterWiseGrades)
            SemesterWiseGrades = []
        else:
            SemesterWiseGrades.append(i)
    
#    print(MasterListCourses)
    
    finalList = []
    for i in range(min(len(MasterListCourses),8)):
        finalList.append(MasterListCourses[i])
        finalList.append(MasterListGrades[i])
    for i in range(len(MasterListCourses),8):
        finalList.append("")
        finalList.append("")
#    print(finalList)
#    print(len(finalList))
    return finalList




# tempdf is the new student_wise data frame
tempDf = pd.DataFrame(columns=["School","Major","Batch","Sem1_courses","Sem1_grades","Sem2_courses","Sem2_grades","Sem3_courses","Sem3_grades","Sem4_courses","Sem4_grades","Sem5_courses","Sem5_grades","Sem6_courses","Sem6_grades","Sem7_courses","Sem7_grades","Sem8_courses","Sem8_grades",])

df = df.reset_index(drop = True)

courseCode = []
gradeObtained = []
for i in range(0,len(df["Year"])-1):
    #add this row's courses and grades to the list for that student
    courseCode.append(df.loc[i][6])
    gradeObtained.append(df.loc[i][8])

    if(df.loc[i][3] != df.loc[i+1][3]):
        courseCode.append("0")
        gradeObtained.append("0")    

    if(df.loc[i][4]>df.loc[i+1][4]):
        tempDf.loc[i] = [df.loc[i][0],df.loc[i][1],df.loc[i][5]]+parseIntoSemesters(courseCode,gradeObtained)
        courseCode=[]
        gradeObtained=[]
print(tempDf.head(0))
print(tempDf)
tempDf = tempDf.reset_index(drop = True)  
tempDf.to_csv('../data/preprocessed/Student_wise_data.csv', sep=',', encoding='utf-8',header=True)

Empty DataFrame
Columns: [School, Major, Batch, Sem1_courses, Sem1_grades, Sem2_courses, Sem2_grades, Sem3_courses, Sem3_grades, Sem4_courses, Sem4_grades, Sem5_courses, Sem5_grades, Sem6_courses, Sem6_grades, Sem7_courses, Sem7_grades, Sem8_courses, Sem8_grades]
Index: []
                               School Major Batch              Sem1_courses  \
20              School of Engineering   EED  2012          [CCC303, CCC302]   
44              School of Engineering   EED  2012          [CCC303, CCC302]   
63              School of Engineering   ECE  2012          [CCC303, CCC302]   
72              School of Engineering   CED  2012          [CCC302, CCC303]   
88     School of Humanities & Soc Sci   ENG  2013          [CCC302, CCC801]   
...                               ...   ...   ...                       ...   
16525  School of Humanities & Soc Sci   SOC  2017                  [CCC302]   
16529      School of Natural Sciences   BIO  2017          [CCC512, CCC302]   
16537      Scho

In [9]:
# making a dataframe df_ccc containing only ccc courses
df_ccc=pd.read_csv("../data/preprocessed/Final_data.csv")
for i,row in df_ccc.iterrows():
    if not row['Course'].startswith('CCC'):
        df_ccc.drop(i,inplace=True)
df_ccc.drop(['Unnamed: 0'],inplace=True,axis=1)  

#making student wise dataframe tempDf_ccc just for ccc courses
tempDf_ccc = pd.DataFrame(columns=["School","Major","Batch","Sem1_courses","Sem1_grades","Sem2_courses","Sem2_grades","Sem3_courses","Sem3_grades","Sem4_courses","Sem4_grades","Sem5_courses","Sem5_grades","Sem6_courses","Sem6_grades","Sem7_courses","Sem7_grades","Sem8_courses","Sem8_grades",])
df_ccc = df_ccc.reset_index(drop = True)
courseCode = []
gradeObtained = []
for i in range(0,len(df_ccc["Year"])-1):
    #add this row's courses and grades to the list for that student
    courseCode.append(df_ccc.loc[i][6])
    gradeObtained.append(df_ccc.loc[i][8])
    if(df_ccc.loc[i][3] != df_ccc.loc[i+1][3]):
        courseCode.append("0")
        gradeObtained.append("0")    

    if(df_ccc.loc[i][4]>df_ccc.loc[i+1][4]):
        tempDf_ccc.loc[i] = [df_ccc.loc[i][0],df_ccc.loc[i][1],df_ccc.loc[i][5]]+parseIntoSemesters(courseCode,gradeObtained)
        courseCode=[]
        gradeObtained=[]
df_ccc=df_ccc.reset_index(drop=True)
tempDf_ccc=tempDf_ccc.reset_index(drop=True)
print(tempDf_ccc.head(0))
print(tempDf_ccc)
tempDf_ccc.to_csv('../data/preprocessed/Student_ccc.csv',sep=',', encoding='utf-8',header=True)
df_ccc.to_csv('../data/preprocessed/Final_ccc.csv',sep=',',encoding='utf-8',header=True)

Empty DataFrame
Columns: [School, Major, Batch, Sem1_courses, Sem1_grades, Sem2_courses, Sem2_grades, Sem3_courses, Sem3_grades, Sem4_courses, Sem4_grades, Sem5_courses, Sem5_grades, Sem6_courses, Sem6_grades, Sem7_courses, Sem7_grades, Sem8_courses, Sem8_grades]
Index: []
                              School Major Batch              Sem1_courses  \
0              School of Engineering   EED  2012          [CCC303, CCC302]   
1              School of Engineering   EED  2012          [CCC303, CCC302]   
2              School of Engineering   ECE  2012          [CCC303, CCC302]   
3              School of Engineering   CED  2012          [CCC302, CCC303]   
4     School of Humanities & Soc Sci   ENG  2013          [CCC302, CCC801]   
...                              ...   ...   ...                       ...   
1376           School of Engineering   CED  2017                  [CCC302]   
1377  School of Humanities & Soc Sci   SOC  2017                  [CCC302]   
1378      School of Natu

In [10]:
# making a dataframe df_uwe containing only uwe courses
df_uwe=pd.read_csv("../data/preprocessed/Final_data.csv")
df_uwe= df_uwe.reset_index(drop = True)
tempDf_uwe=pd.read_csv("../data/preprocessed/Student_wise_data.csv")
tempDf_uwe = tempDf_uwe.reset_index(drop = True)

df_uwe.drop(['Unnamed: 0'],inplace=True,axis=1)
tempDf_uwe.drop(['Unnamed: 0'],inplace=True,axis=1)

for i,row in df_uwe.iterrows():
    if row['Course'].startswith('CCC'):
        df_uwe.drop(i,inplace=True)       
df_uwe = df_uwe.reset_index(drop = True)

# making a student wise dataframe tempDf_uwe just for uwe courses using tempDf

for i in range(0,len(tempDf_uwe)):
    j=3
    while (j<18):
        lst=tempDf_uwe.iloc[i,j]
        lst1=tempDf_uwe.iloc[i,j+1]
        if not str(lst).startswith('nan'):
            lst=lst[1:-1].split(",")
            lst1=lst1[1:-1].split(",")
            k=0
            while k<len(lst):
                if lst[k].strip().startswith('\'CCC'):
                    del lst[k]
                    del lst1[k]
                else:
                    k+=1
            tempDf_uwe.iloc[i,j]=str(lst)
            tempDf_uwe.iloc[i,j+1]=str(lst1)
        j+=2
print(tempDf_uwe.head(0))
print(tempDf_uwe)
tempDf_uwe.to_csv('../data/preprocessed/Student_uwe.csv',sep=',', encoding='utf-8',header=True)
df_uwe.to_csv('../data/preprocessed/Final_uwe.csv',sep=',',encoding='utf-8',header=True)

Empty DataFrame
Columns: [School, Major, Batch, Sem1_courses, Sem1_grades, Sem2_courses, Sem2_grades, Sem3_courses, Sem3_grades, Sem4_courses, Sem4_grades, Sem5_courses, Sem5_grades, Sem6_courses, Sem6_grades, Sem7_courses, Sem7_grades, Sem8_courses, Sem8_grades]
Index: []
                              School Major  Batch                Sem1_courses  \
0              School of Engineering   EED   2012                          []   
1              School of Engineering   EED   2012                          []   
2              School of Engineering   ECE   2012                          []   
3              School of Engineering   CED   2012                          []   
4     School of Humanities & Soc Sci   ENG   2013                          []   
...                              ...   ...    ...                         ...   
1384  School of Humanities & Soc Sci   SOC   2017                          []   
1385      School of Natural Sciences   BIO   2017                          [] 

# 
Out initial raw data was in `Student_Result_Data.csv` and we preprocessed and stored the following csv files

|DataFrame | Preprocessed Csv file| 
|-------|-------|
| df     |  Final_data.csv   | 
| tempDf    | Student_wise_data.csv      | 
| df_ccc    | Final_ccc.csv     | 
|tempDF_ccc|  Student_ccc.csv|
|df_uwe|Final_uwe.csv  |
|tempDf_uwe |Student_uwe.csv |

     